In [69]:
# Install Gradio
!pip install gradio

In [70]:
# import numpy and pandas library
import numpy as np
import pandas as pd


In [71]:
# Read the dataset from github
url = 'https://raw.githubusercontent.com/Honai-Tech/fastTest/main/adverse-effects.csv'
df = pd.read_csv(url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
df.iloc[:,5].unique()

array(['03', '13', '40', '07', '20', '54', '29', '53', '09', '31', '22',
       '45', '35', '37', '12', '04', '21', '24', '05', '41', '34', '23',
       '33', '25', '16', '27', '26', 'Not Available', '02', '17', '15',
       '28', '39', '46', '38', '18', '36', '30', '32', '52', '51', '14',
       '47', '50', 5, 54, 40, 3, 53, 13, 41, 24, 20, 12, 29, 21, 15, 50,
       45, 46, 7, 37, 26, 23, 34, 9, 2, 16, 33, 14, 38, 31, 17, 28, 32,
       25, 4, 35, 30, 27, 39, 18, 22, 36, 42], dtype=object)

In [73]:
df['PRI_FDA Industry Code'] = df['PRI_FDA Industry Code'].map({'Not Available': '00'}).fillna(df['PRI_FDA Industry Code'])

In [74]:
df.iloc[:,5].unique()

array(['03', '13', '40', '07', '20', '54', '29', '53', '09', '31', '22',
       '45', '35', '37', '12', '04', '21', '24', '05', '41', '34', '23',
       '33', '25', '16', '27', '26', '00', '02', '17', '15', '28', '39',
       '46', '38', '18', '36', '30', '32', '52', '51', '14', '47', '50',
       5, 54, 40, 3, 53, 13, 41, 24, 20, 12, 29, 21, 15, 50, 45, 46, 7,
       37, 26, 23, 34, 9, 2, 16, 33, 14, 38, 31, 17, 28, 32, 25, 4, 35,
       30, 27, 39, 18, 22, 36, 42], dtype=object)

In [75]:
df = df[~df['CI_Age Unit'].isin(['Month(s)', 'Not Available', 'Week(s)', 'Day(s)', 'Decade(s)'])]

df = df.fillna(0)
df.isnull().sum()

RA_Report #                        0
RA_CAERS Created Date              0
AEC_Event Start Date               0
PRI_Product Role                   0
PRI_Reported Brand/Product Name    0
PRI_FDA Industry Code              0
PRI_FDA Industry Name              0
CI_Age at Adverse Event            0
CI_Age Unit                        0
CI_Gender                          0
AEC_One Row Outcomes               0
SYM_One Row Coded Symptoms         0
dtype: int64

In [76]:
df['CI_Age Unit'].unique()

array(['Year(s)'], dtype=object)

In [77]:
# Remove the first 4 columns
df1 = df.iloc[:, 4:12]
df1.head()

,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Age Unit,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
0,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,03,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL..."
4,"FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS",07,Snack Food Item,10.0,Year(s),Male,NON-SERIOUS INJURIES/ ILLNESS,CHOKING
8,YOHIMBE,54,Vit/Min/Prot/Unconv Diet(Human/Animal),66.0,Year(s),Male,REQ. INTERVENTION TO PRVNT PERM. IMPRMNT.,BLOOD PRESSURE INCREASED
15,METOBOLITE 356,54,Vit/Min/Prot/Unconv Diet(Human/Animal),51.0,Year(s),Male,DEATH,"COMPLETED SUICIDE, STRESS SYMPTOMS, DEATH"
16,METABOLIFE,54,Vit/Min/Prot/Unconv Diet(Human/Animal),45.0,Year(s),Female,DEATH,"DEATH, MITRAL VALVE INCOMPETENCE"


In [78]:
# Sort the data for only the wanted ones
food_name = df['PRI_Reported Brand/Product Name']
idustry_name = df['PRI_FDA Industry Name']
age = df['CI_Age at Adverse Event']
gender = df['CI_Gender']

df2 = pd.concat([food_name,idustry_name,age,gender],axis=1)
df2.head()

,PRI_Reported Brand/Product Name,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Gender
0,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,Bakery Prod/Dough/Mix/Icing,2.0,Female
4,"FRITO LAY FUNYUNS ONION FLAVOR, ONION RINGS",Snack Food Item,10.0,Male
8,YOHIMBE,Vit/Min/Prot/Unconv Diet(Human/Animal),66.0,Male
15,METOBOLITE 356,Vit/Min/Prot/Unconv Diet(Human/Animal),51.0,Male
16,METABOLIFE,Vit/Min/Prot/Unconv Diet(Human/Animal),45.0,Female


In [79]:
sym_exp = df["SYM_One Row Coded Symptoms"].str.split(pat=",", expand=True)
sym_exp = sym_exp.astype('string')

sym_exp.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52
0,SWELLING FACE,WHEEZING,COUGH,RASH,HOSPITALISATION,DYSPNOEA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,CHOKING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,BLOOD PRESSURE INCREASED,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15,COMPLETED SUICIDE,STRESS SYMPTOMS,DEATH,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
16,DEATH,MITRAL VALVE INCOMPETENCE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [80]:
sym_exp = sym_exp.fillna('None')

In [81]:
# Encode the symptoms
from sklearn.preprocessing import LabelEncoder
sym_enc = pd.DataFrame(columns=sym_exp.columns, data=LabelEncoder().fit_transform(sym_exp.values.flatten()).reshape(sym_exp.shape))

In [82]:
sym_enc.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52
0,4677,3237,826,2559,1541,1004,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
1,3583,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
2,3478,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
3,3620,2869,873,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310
4,3656,1978,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310,4310


In [83]:
# Change the labels of <N\A> to 0
sym_enc = sym_enc.replace(4310, 0)
sym_enc.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52
0,4677,3237,826,2559,1541,1004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3620,2869,873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3656,1978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
sym = sym_enc.dropna()
sym.shape

(46982, 53)

In [85]:
df2['Gender_Encoded'] = LabelEncoder().fit_transform(df2['CI_Gender'])
df2['FoodName_Encoded'] = LabelEncoder().fit_transform(df2['PRI_Reported Brand/Product Name'])
df2['IndustryName_Encoded'] = LabelEncoder().fit_transform(df2['PRI_FDA Industry Name'])

In [86]:
X = pd.merge(df2, sym, left_index=True, right_index=True)
X.drop(columns=[ "CI_Gender","PRI_Reported Brand/Product Name", "FoodName_Encoded","PRI_FDA Industry Name","CI_Age at Adverse Event","IndustryName_Encoded","Gender_Encoded"], axis=1, inplace=True)
df2.drop(columns=[ "CI_Gender","PRI_Reported Brand/Product Name"], axis=1, inplace=True)


In [87]:
X.columns, df2.columns

(Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52],
       dtype='object'),
 Index(['PRI_FDA Industry Name', 'CI_Age at Adverse Event', 'Gender_Encoded',
        'FoodName_Encoded', 'IndustryName_Encoded'],
       dtype='object'))

In [88]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52
0,4677,3237,826,2559,1541,1004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3656,1978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,4374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,4389,2379,1541,2272,953,694,2233,1138,3172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,4389,2379,1541,2272,2233,694,1138,953,3172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
# Specify input and output data for model training
x_col = X.columns
master_df = pd.merge(X, df2, left_index=True, right_index=True)
XX = master_df.drop(columns=["FoodName_Encoded","PRI_FDA Industry Name",'IndustryName_Encoded'], axis=1)
# yy = master_df['FoodName_Encoded']
yy = master_df['IndustryName_Encoded']


In [90]:
master_df.columns

Index([                        0,                         1,
                               2,                         3,
                               4,                         5,
                               6,                         7,
                               8,                         9,
                              10,                        11,
                              12,                        13,
                              14,                        15,
                              16,                        17,
                              18,                        19,
                              20,                        21,
                              22,                        23,
                              24,                        25,
                              26,                        27,
                              28,                        29,
                              30,                        31,
                        

In [91]:
XX.shape, yy.shape

((24574, 55), (24574,))

In [92]:
XX.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,CI_Age at Adverse Event,Gender_Encoded
0,4677,3237,826,2559,1541,1004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0
4,3656,1978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.0,1
8,4374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66.0,1
15,4389,2379,1541,2272,953,694,2233,1138,3172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.0,1
16,4389,2379,1541,2272,2233,694,1138,953,3172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45.0,0


In [93]:
yy.head()

0      2
4     30
8     37
15    37
16    37
Name: IndustryName_Encoded, dtype: int64

In [94]:
# Split the data into tranining set and the test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(XX, yy, test_size = 0.1)


In [95]:
# Train the model using ML models
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

classifier = RandomForestClassifier()
# classifier = DecisionTreeClassifier()

classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [96]:
# Predict the test set
y_pred = classifier.predict(X_test)
y_pred
y_pred.shape

(2458,)

In [97]:
foo = LabelEncoder().fit(df2['PRI_FDA Industry Name'])

prediction_decoded = foo.inverse_transform(y_pred)
print(prediction_decoded)


['Vit/Min/Prot/Unconv Diet(Human/Animal)'
 'Vit/Min/Prot/Unconv Diet(Human/Animal)' 'Nuts/Edible Seed' ...
 'Vit/Min/Prot/Unconv Diet(Human/Animal)'
 'Vit/Min/Prot/Unconv Diet(Human/Animal)'
 'Vit/Min/Prot/Unconv Diet(Human/Animal)']


In [98]:
# Cek model accuracy
from sklearn.metrics import accuracy_score
print("Accuracy of Model::",accuracy_score(y_test,y_pred))


Accuracy of Model:: 0.6891781936533767


In [99]:
# Deploy the model using Gradio
import gradio as gr

s = sym_exp.iloc[:,0].dropna()
s.columns = ['Symptom']
# choices = list(s.head(50))

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

from sklearn.preprocessing import LabelEncoder
sym_exp = sym_exp.fillna('None')
sym_single = pd.unique(sym_exp.values.ravel('K'))
sym_single_data = pd.DataFrame(sym_single)

sym_single_enc = LabelEncoder().fit_transform(sym_single_data)

le.fit(['Male','Female'])

le2 = LabelEncoder().fit(sym_single_data)
sym_single.sort()
choices = list(sym_single)


def food(age,gender,symptoms,symptoms2,symptoms3):

  g_enc = le.transform([gender])
  s_enc = le2.transform([symptoms])
  s_enc2 = le2.transform([symptoms2]) 
  s_enc3 = le2.transform([symptoms3]) 

    
  di = pd.DataFrame(np.zeros((1, 55)))


  di[0] = s_enc
  di[1] = s_enc2
  di[2] = s_enc3
  di[53] = age
  di[54] = g_enc
    
  do = classifier.predict(di)
  do_decoded = foo.inverse_transform(do)  
  
  do2 = classifier.predict(di)
  do_decoded2 = foo.inverse_transform(do2) 

  
  return do_decoded[0]


age = gr.inputs.Textbox("text")
gender = gr.inputs.Dropdown(['Male','Female'])
symptoms = gr.inputs.Dropdown(choices)
symptoms2 = gr.inputs.Dropdown(choices)
symptoms3 = gr.inputs.Dropdown(choices)
out = gr.outputs.Textbox(label='Food to be avoided')

# out2 = gr.outputs.Textbox(label='Food to be avoided 2')


gr.Interface(food, inputs=[age, gender, symptoms, symptoms2, symptoms3], outputs = [out]).launch(debug=False)




/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://54654.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://54654.gradio.app')